In [1]:
import pandas as pd
import mlflow
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
from sklearn.metrics import root_mean_squared_error

In [2]:
taxi_zones = pd.read_csv("../data/taxi_zone_lookup_coordinates.csv")

In [3]:
def preprocess(data):
    data['duration'] = data['lpep_dropoff_datetime'] - data['lpep_pickup_datetime']
    data['duration'] = data['duration'].apply(lambda x: x.total_seconds() / 60)
    data = data[(data['duration'] >= data['duration'].quantile(0.04)) & (data['duration'] <= data['duration'].quantile(0.98))]
    categorical = ['trip_type', 'PULocationID', 'DOLocationID']
    numerical = ['trip_distance']
    target = ['duration']
    data = data[numerical + categorical + target]
    data['trip_type'].fillna(data.trip_type.mode().item(), inplace=True)
    data = pd.merge(data, taxi_zones[['LocationID', 'latitude', 'longitude']], left_on='PULocationID', right_on='LocationID', how='left').rename(columns={"latitude": "PULat",
                                                                                                                                                   "longitude": "PULon"})
    data = pd.merge(data, taxi_zones[['LocationID', 'latitude', 'longitude']], left_on='DOLocationID', right_on='LocationID', how='left').rename(columns={"latitude": "DOLat",
                                                                                                                                                   "longitude": "DOLon"})
    data = data.drop(columns=["PULocationID", "DOLocationID", "LocationID_x", "LocationID_y"])
    data["PULat"].fillna(data["PULat"].mean(), inplace=True)
    data["PULon"].fillna(data["PULon"].mean(), inplace=True)
    data["DOLat"].fillna(data["DOLat"].mean(), inplace=True)
    data["DOLon"].fillna(data["DOLon"].mean(), inplace=True)
    onehot = pd.get_dummies(data['trip_type'], prefix="triptype").map(lambda x: 1 if x else 0)
    data = data.join(onehot).drop(columns=['trip_type'])
    data = data[(data['trip_distance'] >= data['trip_distance'].quantile(0.04)) & (data['trip_distance'] < data['trip_distance'].quantile(0.99))]
    
    for column in ['PULat', 'PULon', 'DOLat', "DOLon"]:
        data[column] = (data[column] - data[column].mean()) / data[column].std()

    for column in ['trip_distance', 'duration']:
        data[column] = (data[column] - data[column].min()) / (data[column].max() - data[column].min())
    
    return data[['PULat', 'PULon', 'DOLat', "DOLon", "trip_distance"]], data['duration']

In [6]:
train_data = pd.read_parquet("/workspaces/taxiML/data/green_tripdata_2021-01.parquet")
val_data = pd.read_parquet("/workspaces/taxiML/data/green_tripdata_2021-02.parquet")
x_train, y_train = preprocess(train_data)
x_val, y_val = preprocess(val_data)
train = xgb.DMatrix(x_train, label=y_train)
valid = xgb.DMatrix(x_val, label=y_val)

/tmp/ipykernel_15276/4188487381.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['trip_type'].fillna(data.trip_type.mode().item(), inplace=True)
/tmp/ipykernel_15276/4188487381.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(valu

In [4]:
mlflow.set_tracking_uri("sqlite:///../mlflow.db")
mlflow.set_experiment("tune_taxi_experiment")

<Experiment: artifact_location='/workspaces/taxiML/02/mlruns/2', creation_time=1737130798101, experiment_id='2', last_update_time=1737130798101, lifecycle_stage='active', name='tune_taxi_experiment', tags={}>

In [5]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [11]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/codespace/.python/current/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:25:27] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:0.17832                           
[1]	validation-rmse:0.16080                           
[2]	validation-rmse:0.14674                           
[3]	validation-rmse:0.13557                           
[4]	validation-rmse:0.12693                           
[5]	validation-rmse:0.12026                           
[6]	validation-rmse:0.11516                           
[7]	validation-rmse:0.11137                           
[8]	validation-rmse:0.10861                           
[9]	validation-rmse:0.10656                           
[10]	validation-rmse:0.10508                          
[11]	validation-rmse:0.10403                          
[12]	validation-rmse:0.10335                          
[13]	validation-rmse:0.10289                          
[14]	validation-rmse:0.10261                          
[15]	validation-rmse:0.10241                          
[16]	validation-rmse:0.10228                          
[17]	validation-rmse:0.10224                          
[18]	valid

/home/codespace/.python/current/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:25:34] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:0.17369                                                      
[1]	validation-rmse:0.15423                                                      
[2]	validation-rmse:0.13984                                                      
[3]	validation-rmse:0.12980                                                      
[4]	validation-rmse:0.12232                                                      
[5]	validation-rmse:0.11747                                                      
[6]	validation-rmse:0.11383                                                      
[7]	validation-rmse:0.11149                                                      
[8]	validation-rmse:0.10993                                                      
[9]	validation-rmse:0.10892                                                      
[10]	validation-rmse:0.10820                                                     
[11]	validation-rmse:0.10775                                                     
[12]	validation-

...

[5]	validation-rmse:0.14836                                                      
[6]	validation-rmse:0.14246                                                      
[7]	validation-rmse:0.13729                                                      
[8]	validation-rmse:0.13263                                                      
[9]	validation-rmse:0.12856                                                      
[10]	validation-rmse:0.12494                                                     
[11]	validation-rmse:0.12176                                                     
[12]	validation-rmse:0.11894                                                     
[13]	validation-rmse:0.11648                                                     
[14]	validation-rmse:0.11433                                                     
[15]	validation-rmse:0.11246                                                     
[16]	validation-rmse:0.11084                                                     
[17]	validation-

/home/codespace/.python/current/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:31:23] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[5]	validation-rmse:0.10401                                                      
[6]	validation-rmse:0.10327                                                      
[7]	validation-rmse:0.10301                                                      
[8]	validation-rmse:0.10287                                                      
[9]	validation-rmse:0.10290                                                      
[10]	validation-rmse:0.10298                                                     
[11]	validation-rmse:0.10312                                                     
[12]	validation-rmse:0.10323                                                     
[13]	validation-rmse:0.10334                                                     
[14]	validation-rmse:0.10340                                                     
[15]	validation-rmse:0.10344                                                     
[16]	validation-rmse:0.10353                                                     
[17]	validation-

/home/codespace/.python/current/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:31:26] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[4]	validation-rmse:0.10228                                                      
[5]	validation-rmse:0.10243                                                      
[6]	validation-rmse:0.10261                                                      
[7]	validation-rmse:0.10292                                                      
[8]	validation-rmse:0.10314                                                      
[9]	validation-rmse:0.10334                                                      
[10]	validation-rmse:0.10348                                                     
[11]	validation-rmse:0.10355                                                     
[12]	validation-rmse:0.10380                                                     
[13]	validation-rmse:0.10392                                                     
[14]	validation-rmse:0.10409                                                     
[15]	validation-rmse:0.10415                                                     
[16]	validation-

In [7]:
mlflow.xgboost.autolog(disable=True)

In [12]:
with mlflow.start_run():
    
    train = xgb.DMatrix(x_train, label=y_train)
    valid = xgb.DMatrix(x_val, label=y_val)

    best_params = {
        'learning_rate': 0.055460798061846325,
        'max_depth': 75,
        'min_child_weight': 11.184212926679496,
        'objective': 'reg:linear',
        'reg_alpha': 0.3647654946314019,
        'reg_lambda': 0.004247458594356321,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact("../data/green_tripdata_2021-01.parquet", artifact_path="training_data")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[0]	validation-rmse:0.19172
[1]	validation-rmse:0.18407
[2]	validation-rmse:0.17691
[3]	validation-rmse:0.17032


/home/codespace/.python/current/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [19:16:46] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[4]	validation-rmse:0.16418
[5]	validation-rmse:0.15859
[6]	validation-rmse:0.15334
[7]	validation-rmse:0.14851
[8]	validation-rmse:0.14407
[9]	validation-rmse:0.14002
[10]	validation-rmse:0.13625
[11]	validation-rmse:0.13285
[12]	validation-rmse:0.12973
[13]	validation-rmse:0.12683
[14]	validation-rmse:0.12421
[15]	validation-rmse:0.12184
[16]	validation-rmse:0.11968
[17]	validation-rmse:0.11772
[18]	validation-rmse:0.11594
[19]	validation-rmse:0.11433
[20]	validation-rmse:0.11288
[21]	validation-rmse:0.11156
[22]	validation-rmse:0.11037
[23]	validation-rmse:0.10926
[24]	validation-rmse:0.10829
[25]	validation-rmse:0.10739
[26]	validation-rmse:0.10659
[27]	validation-rmse:0.10588
[28]	validation-rmse:0.10525
[29]	validation-rmse:0.10467
[30]	validation-rmse:0.10415
[31]	validation-rmse:0.10369
[32]	validation-rmse:0.10328
[33]	validation-rmse:0.10291
[34]	validation-rmse:0.10258
[35]	validation-rmse:0.10229
[36]	validation-rmse:0.10203
[37]	validation-rmse:0.10180
[38]	validation-rmse

/home/codespace/.python/current/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [19:16:52] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/01/17 19:16:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [7]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlmodel = model_class()
        mlmodel.fit(x_train, y_train)

        y_pred = mlmodel.predict(x_val)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

2025/01/17 19:35:52 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 0.24.1 <= scikit-learn <= 1.5.2, but the installed version is 1.6.0. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.
2025/01/17 19:37:10 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpzjuxw6xg/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.6.0', 'cloudpickle==3.1.1']. Set logging level to DEBUG to see the full traceback. 
/home/codespace/.local/lib/python3.12/site-packages/sklearn/svm/_base.py:1243: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
